In [1]:
!pip install langchain
!pip install boto3
!pip install langchain-community
!pip install langchain-aws
!pip install gpt4all
!pip install langchain-qdrant

  Using cached gpt4all-2.8.2-py3-none-manylinux1_x86_64.whl.metadata (4.8 kB)
Using cached gpt4all-2.8.2-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached langchain_qdrant-0.2.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached qdrant_client-1.13.2-py3-none-any.whl.metadata (10 kB)
  Using cached grpcio_tools-1.70.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.3 kB)
  Using cached portalocker-2.10.1-py3-none-any.whl.metadata (8.5 kB)
  Using cached protobuf-5.29.3-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
  Using cached grpcio-1.70.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.9 kB)
Using cached langchain_qdrant-0.2.0-py3-none-any.whl (23 kB)
Using cached qdrant_client-1.13.2-py3-none-any.whl (306 kB)
Using cached grpcio_tools-1.70.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.5 MB)
Using cached grpcio-1.70.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.9 MB)
Using cached portal

In [2]:
import os

import boto3
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.gpt4all import GPT4AllEmbeddings
from langchain_aws.llms.bedrock import BedrockLLM
from langchain.chains import RetrievalQA
from langchain_qdrant import QdrantVectorStore
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
def get_bedrock_runtime(region_name, *args, **kwargs):
    return boto3.client(service_name='bedrock-runtime', region_name=region_name, *args, **kwargs)


def get_langchain_bedrock_llm(model_id, client, *args, **kwargs):
    return BedrockLLM(model_id=model_id, client=client, *args, **kwargs)

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=900,
    chunk_overlap=20,
    length_function=len
)

In [6]:
embed_model_name = 'all-MiniLM-L6-v2.gguf2.f16.gguf'
qdrant_url = os.getenv('QDRANT_URL')
qdrant_api_key = os.getenv('QDRANT_API_KEY')
llm_name = 'mistral.mixtral-8x7b-instruct-v0:1'

In [7]:
aws_access_token = os.getenv('AWS_ACCESS_TOKEN')
aws_secret_key = os.getenv('AWS_ACCESS_KEY')

config = {
      "max_tokens": 4096,
      "temperature": 0.7,
      "top_p": 0.8
    }

bedrock_runtime = get_bedrock_runtime('us-east-1', aws_access_key_id=aws_access_token, aws_secret_access_key=aws_secret_key)
llm = get_langchain_bedrock_llm(llm_name, bedrock_runtime)

In [8]:
embedding_model = GPT4AllEmbeddings(model_name=embed_model_name)

Failed to load libllamamodel-mainline-cuda-avxonly.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory
Failed to load libllamamodel-mainline-cuda.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory


In [9]:
embedding_model.embed_query('What is deep learning?')

[-0.07789280265569687,
 -0.03325372561812401,
 0.019731666892766953,
 0.007232266012579203,
 -0.015520625747740269,
 -0.022012833505868912,
 -0.030771594494581223,
 -0.009376595728099346,
 -0.026375045999884605,
 -0.03906415402889252,
 -0.10199055820703506,
 0.0190118420869112,
 -0.03830576688051224,
 -0.024462716653943062,
 -0.11994853615760803,
 -0.0026534167118370533,
 0.003752821823582053,
 0.045944880694150925,
 -0.13237416744232178,
 -0.05502625182271004,
 0.025544865056872368,
 0.05282978713512421,
 -0.05155644193291664,
 -0.024949990212917328,
 0.05971359834074974,
 0.03389402851462364,
 0.023386601358652115,
 -0.04147306829690933,
 0.016460128128528595,
 -0.011059910990297794,
 0.08573001623153687,
 -0.030412552878260612,
 0.024305101484060287,
 0.06547030061483383,
 -0.04821261763572693,
 0.11049804836511612,
 -0.07268178462982178,
 0.013661586679518223,
 0.012405565939843655,
 -0.027723992243409157,
 -0.03888094797730446,
 -0.024924060329794884,
 -0.0013083000667393208,
 0.0

In [11]:
qdrant = QdrantVectorStore.from_existing_collection(
    embedding=embedding_model,
    url=qdrant_url,
    api_key=qdrant_api_key,
    collection_name='mlbooks',
    prefer_grpc=True
)

retriever = qdrant.as_retriever()
qa = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever, return_source_documents=True)

In [12]:
def convert_langchaindoc_to_dict(doc):
  return {
      'page_content': doc.page_content,
      'metadata': doc.metadata
  }

In [13]:
question = "What is supervised learning?"
result_json = qa.invoke({'query': question})

In [14]:
result_json['result']

' Supervised learning is a type of machine learning where the algorithm is provided with a set of labeled training data. This means that the data used to train the model has been tagged with the correct output. For example, in a document classification task, the documents are already labeled with their correct classification. The model learns to predict the correct output based on the input data and the labeled examples it was trained on.'